In [6]:
%load_ext autoreload
%autoreload 2

import os
import random
import sys

from pathlib import Path
from hydra import initialize_config_dir, compose
from hydra.utils import instantiate
from omegaconf import OmegaConf
import shutil
import tqdm
import torch
import matplotlib.pyplot as plt

RESULTS_DIR = "singles"
NUM_REPEATS = 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# New single results
session_ids = list(os.listdir(RESULTS_DIR))

results = {}
for sid in session_ids:
    rsum = 0
    for nidx in range(NUM_REPEATS):
        sf = os.path.join(RESULTS_DIR, sid, f"results_{nidx}.torch")
        _r = torch.load(sf, map_location=torch.device('cpu'))
        rsum += _r["final_test_r2"]
    results[sid] = rsum / NUM_REPEATS

In [13]:
# Old results
infile = "/home/mmattb/Projects/opto-coproc/multisession_st_dist.torch"
old_results = torch.load(infile, map_location=torch.device('cpu'))

def parse_results(results):
    dist = []
    disttrend = []
    dist_test = []
    disttrend_test = []
    dist_loss = []
    dist_loss_test = []
    per_session_train_r2s = {}
    per_session_test_r2s = {}
    per_session_test_losses = {}
    for k, result in results.items():
        per_session_test_losses[k] = []
        for sr in result:
            dist.append(max(sr[-5]))
            dist_test.append(max([rr[-1] for rr in sr[-2]]))

            disttrend.append(sr[6])
            disttrend_test.append(sr[7])
    
            dist_loss.append(min(sr[2]))
            dist_loss_test.append(min(sr[3]))
    
            per_session_train_r2s[k] = dist[-1]
            per_session_test_r2s[k] = dist_test[-1]
    
            per_session_test_losses[k].append(dist_loss_test[-1])

    return per_session_test_r2s

old_results = parse_results(old_results)

In [15]:
for sid, new_tr2 in results.items():
    old_tr2 = old_results[sid]
    print(sid, old_tr2, new_tr2)

# Bad....

MonkeyJ_20160426_Session2_S1 0.5786245465278625 0.282552828391393
MonkeyJ_20160429_Session3_S1 0.1695532351732254 0.013659000707169374
MonkeyJ_20160426_Session1_S1 0.45400190353393555 0.005016507037604849
MonkeyG_20150917_Session1_M1 0.2445678561925888 0.12009593347708385
MonkeyG_20150921_Session5_S1 0.40744584798812866 0.22286629180113474
MonkeyG_20150921_Session3_S1 0.382977157831192 0.25498584906260174
MonkeyJ_20160426_Session3_S1 0.5296066999435425 0.25653498371442157
MonkeyG_20150925_Session1_S1 0.27470555901527405 -0.15962793429692587
MonkeyJ_20160625_Session5_S1 0.7609798312187195 0.27035797635714215
MonkeyJ_20160627_Session2_S1 0.5739260911941528 0.31615303953488666
MonkeyG_20150917_Session2_M1 0.23392876982688904 0.061992151041825615
MonkeyJ_20160702_Session4_S1 0.0910683423280716 0.04572491596142451
MonkeyJ_20160627_Session1_S1 0.5198116898536682 0.3855733374754588
MonkeyG_20150916_Session4_S1 0.7518243789672852 0.6735852559407552
MonkeyG_20150918_Session1_S1 0.69630956649780